## If you use this script for your research, please cite our paper: Ciancetta A, Gill A. K., Ding T., Karlov D.S., McCormick P., Tikhonova I.G. 'Probe Confined Dynamic Mapping for GPCRs Allosteric Site Prediction' eLife, 2021, Submitted.

## Initialize HTMD

In [1]:
from htmd.ui import *
from htmd.molecule.util import uniformRandomRotation
import nglview as nv
from nglview.datafiles import PDB, XTC
import configparser
from itertools import groupby
import math



Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. 
https://dx.doi.org/10.1021/acs.jctc.6b00049
Documentation: http://software.acellera.com/
To update: conda update htmd -c acellera -c psi4

New devel HTMD version (1.12.2 python[3.6,<3.7.0a0,3.5,<3.6.0a0]) is available. You are currently on (1.9.10). Use 'conda update -c acellera htmd' to update to the new version. You might need to update your python version as well if there is no release for your current version.

Deprecation warning: "from htmd import *" will be deprecated. 
To import all HTMD shortcuts please from now on use "from htmd.ui import *"


## Load variables from Input File

In [2]:
cfg = configparser.RawConfigParser()
cfg.read('midas_acemd.cfg')      
par=dict(cfg.items("Settings"))
for p in par:
    par[p]=par[p].split("#",1)[0].strip() # To get rid of inline comments
globals().update(par)  #Make them availible globally
print (par)

{'name': 'M2_oriented', 'ligname': 'iperoxo', 'lig': 'LIG', 'lipid': 'POPC', 'coname': 'Thieno-23-pyridine', 'pm': '135.18', 'conc': '10', 'csolv': 'THP', 'zoff': '6.000000', 'zheight': '20.0000', 'offset': '1.000000', 'zoffset': '1.000000', 'disu': '96 - 176; 413 - 416'}


## Chek if Packmol Runs

In [3]:
pmol = shutil.which("packmol", mode=os.X_OK)
if not pmol:
    raise FileNotFoundError('packmol not found. You should have packmol installed ')

## Calculate Cosolvent Box Dimensions

In [6]:
mol1 = Molecule(name + '.pdb')
#Create variable with protein coordinates
#c=mol1.get('coords')
c=mol1.get('coords', sel='protein and z > 0')
#Calculate minimum and maximum coordinates
pmin=np.min(c, axis=0)
pmax=np.max(c, axis=0)
xmin=pmin[0] 
xmax=pmax[0] 
ymin=pmin[1]
ymax=pmax[1] 
zmin=pmin[2]
zmax=pmax[2]
xsize=xmax-xmin 
ysize=ymax-ymin
zl2=zmax + float(zoff)
zl4=zl2+ float(zheight)
print(xsize,ysize,zl2,zl4)

48.734 33.959 32.308000564575195 52.308000564575195


## Visualize box

In [5]:
mol1.view('not water', gui=False)
b = VMDBox([xmin, xmax, ymin, ymax, zl2, zl4])

## Calculate number of water molecules fitting the box

In [7]:
#Create a box of soley water molecules
smol1 = solvate(mol1, minmax=[[xmin, ymin, zl2],[xmax, ymax, zl4]])
smol1.view()
#Calculate the number of water molecules fitting the box
numwat=len(smol1.get("resid",sel="name OH2"))
print(numwat)

2018-05-08 17:23:59,004 - htmd.builder.solvate - INFO - Using water pdb file at: /usr/local/miniconda3/lib/python3.6/site-packages/htmd/builder/wat.pdb
2018-05-08 17:23:59,489 - htmd.builder.solvate - INFO - Replicating 1 water segments, 1 by 1 by 1


Solvating: 100% (1/1) [############################################] eta --:-- -

2018-05-08 17:23:59,818 - htmd.builder.solvate - INFO - 1016 water molecules were added to the system.



1016


## Calculate number of cosolvent molecules

In [8]:
numsol=numwat*18.0528*(int(conc)/100)/float(pm)
cnumsol=int(math.ceil(numsol))
print(cnumsol)

14


## Calculate Packmol boxes dimensions

In [9]:
intxs=int(math.ceil(float(xsize)))
intys=int(math.ceil(float(ysize)))
intzs=int(math.ceil(float(zheight)))
print("Boxes dimensions:", intxs, intys, intzs)

Boxes dimensions: 49 34 20


## Creates Input for Packmol

In [10]:
output1=conc + "_" + coname + "_box"
packs=output1 + ".inp"
print("""
tolerance 2.5
filetype pdb
output %s.pdb
structure %s.pdb
  number %s
  inside box 0. 0. 0. %s. %s. %s.
end structure
""" %(output1,coname,cnumsol,intxs, intys, intzs), file=open(packs, "w"))

## Runs packmol and creates cosolvent box at desired concentration

In [11]:
os.system(pmol + '<' + packs)

0

## Moves the box to the correct coordinates

In [12]:
cbox=Molecule(output1 + ".pdb")
movedb=cbox.copy()
movedb.moveBy([xmin, ymin , zl2])
movedb.view()
b = VMDBox([xmin, xmax, ymin, ymax, zl2, zl4])
movedb.write(output1 + "_moved.pdb")
cosolvent=(output1 + "_moved.pdb")
print(cosolvent)

10_Thieno-23-pyridine_box_moved.pdb


## System Setup

In [14]:
# System Setup
prot=Molecule(name + '.pdb')  #needs CYS HG->HG1
prot.set('name','HG1',sel='resname CYS and name HG')
#memb=Molecule('step4_lipid.pdb')
memb=Molecule('membrane_c36.pdb')
memb.filter('resname %s' %(lipid))  
prot.set('segid', 'PRO', sel='protein')
prot.set('segid', 'W', sel='water')
prot.set('segid', 'CA', sel='resname CA')
prot.remove('resname ACE')
prot.remove('resname NMA')
#Add ligand in the receptor
lig = Molecule(ligname + '.pdb')
lig.set('segid', 'LIG')
prot.append(lig)
#Add cosolvent box
csol = Molecule(cosolvent)
csol.set('segid','C1')
prot.append(csol)
mol2=embed(prot,memb)
mol2.view()
mol2.write("embedded_system.pdb")

2018-05-08 17:26:03,394 - htmd.molecule.molecule - INFO - Removed 10611 atoms. 28810 atoms remaining in the molecule.
2018-05-08 17:26:03,504 - htmd.molecule.molecule - INFO - Removed 0 atoms. 4556 atoms remaining in the molecule.
2018-05-08 17:26:03,532 - htmd.molecule.molecule - INFO - Removed 0 atoms. 4556 atoms remaining in the molecule.


## System Solvation

In [15]:
smol2 = solvate(mol2, negz=30, posz=10)
smol2.write('solvated_system.pdb')
smol2.view()

2018-05-08 17:26:18,899 - htmd.builder.solvate - INFO - Using water pdb file at: /usr/local/miniconda3/lib/python3.6/site-packages/htmd/builder/wat.pdb
2018-05-08 17:26:19,529 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2


Solvating: 100% (8/8) [############################################] eta 00:00 \


2018-05-08 17:26:28,415 - htmd.builder.solvate - INFO - 23831 water molecules were added to the system.


## Print Lig e Csolv Par and Top Filenames

In [16]:
ligt="top_" + ligname + ".rtf"
ligp="par_" + ligname + ".prm"
csolt="top_" + coname + ".rtf"
csolp="par_" + coname + ".prm"
print(ligt, ligp, csolt, csolp)

top_iperoxo.rtf par_iperoxo.prm top_Thieno-23-pyridine.rtf par_Thieno-23-pyridine.prm


## Create PSF File

In [17]:
ds = [ DisulfideBridge('P',96,'P',176), DisulfideBridge('P',413,'P',416)] ##M2 receptor
topos  = [ligt, csolt,'top/top_all36_prot.rtf','top/top_all36_cgenff.rtf','top/top_all36_lipid.rtf','top/top_water_ions.rtf']
params = [ligp, csolp,'par/par_all36_prot.prm','par/par_all36_cgenff.prm','par/par_all36_lipid.prm','par/par_water_ions.prm']
plm = charmm.build(smol2, topo=topos, param=params,outdir='build',ionize=True,saltconc=0.154,saltanion='CLA',saltcation='SOD')

2018-05-08 17:26:42,666 - htmd.builder.charmm - INFO - Writing out segments.
2018-05-08 17:26:50,567 - htmd.builder.builder - INFO - 2 disulfide bonds were added


Bond between A: [serial 25560 resid 413 resname CYS chain A segid PRO]
             B: [serial 25594 resid 416 resname CYS chain A segid PRO]

Bond between A: [serial 22955 resid 96 resname CYS chain A segid PRO]
             B: [serial 24250 resid 176 resname CYS chain A segid PRO]



2018-05-08 17:26:50,769 - htmd.builder.charmm - INFO - Starting the build.
2018-05-08 17:26:51,481 - htmd.builder.charmm - WARNING - Failed to set coordinates for 4 atoms.
2018-05-08 17:26:51,483 - htmd.builder.charmm - WARNING - Failed to guess coordinates for 1 atoms due to bad angles.
2018-05-08 17:26:51,484 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 13 atoms.
2018-05-08 17:26:51,485 - htmd.builder.charmm - WARNING - Please check /home/antonella/20180508_M2_THP_Ago_New/build/log.txt for further information.
2018-05-08 17:26:51,486 - htmd.builder.charmm - INFO - Finished building.
2018-05-08 17:26:54,418 - htmd.builder.ionize - INFO - Adding 12 anions + 0 cations for neutralizing and 138 ions for the given salt concentration.
2018-05-08 17:26:54,752 - htmd.builder.ionize - INFO - Min distance of ions from molecule: 5A
2018-05-08 17:26:54,753 - htmd.builder.ionize - INFO - Min distance between ions: 5A
2018-05-08 17:26:54,754 - htmd.builder.ionize - INFO - Placin

## View System

In [18]:
plm.view()

## Measure Cell Dimensions

In [19]:
boxc=plm.get('coords')
bpmin=np.min(boxc, axis=0)
bpmax=np.max(boxc, axis=0)
bxmin=bpmin[0] 
bxmax=bpmax[0] 
bymin=bpmin[1]
bymax=bpmax[1] 
bzmin=bpmin[2]
bzmax=bpmax[2]
bxsize=bxmax-bxmin 
bysize=bymax-bymin
bzsize=bzmax-bzmin
print(bxsize,bysize,bzsize)

92.906 93.597 126.022


## Measure Cell Origin

In [20]:
bcen=np.average(boxc, axis=0)
print(bcen)
bcenx=bcen[0]
bceny=bcen[1]
bcenz=bcen[2]
print(bcenx,bceny,bcenz)

[-4.139481  -2.5611055 -3.8518307]
-4.139481 -2.5611055 -3.8518307


## Measure Protein Mass Center

In [21]:
protc=plm.get('coords', sel='protein')
protm=plm.get('masses', sel='protein')
print(protm)
protcm=np.average(protc, axis=0, weights=protm)
print(protcm)
protcenx=protcm[0]
protceny=protcm[1]
protcenz=protcm[2]
print(protcenx,protceny,protcenz)

[12.011  1.008  1.008 ...  1.008  1.008  1.008]
[-0.5908355   0.15096302 -5.183493  ]
-0.5908355 0.15096302 -5.183493


## Create Constraints for Eq01

In [26]:
print(lig)
cfg = configparser.RawConfigParser()
cfg.read('midas_acemd.cfg')       
par=dict(cfg.items("Settings"))
for p in par:
    par[p]=par[p].split("#",1)[0].strip() # To get rid of inline comments
globals().update(par)  #Make them availible globally
print (par)
print(lig)

Molecule with 31 atoms and 1 frames
Atom field - altloc shape: (31,)
Atom field - atomtype shape: (31,)
Atom field - beta shape: (31,)
Atom field - chain shape: (31,)
Atom field - charge shape: (31,)
Atom field - coords shape: (31, 3, 1)
Atom field - element shape: (31,)
Atom field - insertion shape: (31,)
Atom field - masses shape: (31,)
Atom field - name shape: (31,)
Atom field - occupancy shape: (31,)
Atom field - record shape: (31,)
Atom field - resid shape: (31,)
Atom field - resname shape: (31,)
Atom field - segid shape: (31,)
Atom field - serial shape: (31,)
angles shape: (0, 3)
bonds shape: (68, 2)
bondtype shape: (0,)
box shape: (3, 1)
boxangles shape: (3, 1)
crystalinfo: {'a': 59.032, 'b': 77.399, 'c': 163.822, 'alpha': 90.0, 'beta': 90.0, 'gamma': 90.0, 'sGroup': ['P', '21', '21', '21'], 'z': 4}
dihedrals shape: (0, 4)
fileloc shape: (1, 2)
impropers shape: (0, 4)
reps: 
ssbonds shape: (0,)
step shape: (1,)
time shape: (1,)
topoloc: /home/antonella/20180508_M2_THP_Ago_New/ip

In [27]:
plm.set('beta', '1', sel='resname %s or resname %s or water or ions or protein or (resname %s and name P O11 O12 O13 O14 C11 C12 C13 C14 C15 H11A H11B H12A H12B H13A H13B H13C H14A H14B H14C H15A H15B H15C)' %(lig,csolv,lipid))
plm.write('mineq-01.pdb')

## Create Constraints for Eq02

In [28]:
plm.set('beta', '0', sel='all')
plm.set('beta', '1', sel='resname %s or protein' %(lig))
plm.write('mineq-02.pdb')

## Create Constraints for Prod

In [29]:
plm.set('beta', '0', sel='all')
plm.set('beta', '1', sel='protein and name CA and z > -5 and z < 5')
plm.write('prod.pdb')

## Create Input File for Eq01

In [30]:
print("""
#############################################################
## JOB DESCRIPTION                                         ##
#############################################################

# Min. and Eq. of Co-Solvent Membrane System
# embedded in POPC membrane, ions and water.
# Melting lipid tails. PME, Constant Volume.

#############################################################
## ADJUSTABLE PARAMETERS                                   ##
#############################################################

structure          structure.psf
coordinates        structure.pdb
outputName         mineq-01

set temperature    310

firsttimestep      0

#############################################################
## SIMULATION PARAMETERS                                   ##
#############################################################
# Input
paraTypeCharmm	    on
parameters          parameters

# NOTE: Do not set the initial velocity temperature if you 
# have also specified a .vel restart file!
temperature         $temperature

# Periodic Boundary Conditions
# NOTE: Do not set the periodic cell basis if you have also 
# specified an .xsc restart file!
if {1} { 
cellBasisVector1     %s    0.   0.
cellBasisVector2     0.    %s   0.
cellBasisVector3     0.    0.   %s
cellOrigin           %s %s %s
}
wrapWater           on
wrapAll             on

# Force-Field Parameters
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5

# Integrator Parameters
timestep            1.0  ;# 1fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2  
stepspercycle       20

#PME (for full-system periodic electrostatics)
if {1} {
PME                yes
PMEGridSpacing     1.0
}

# Constant Temperature Control
langevin            on    ;# do langevin dynamics
langevinDamping     1     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature

restartfreq        1000     ;# 1000steps = every 2ps
dcdfreq            1000
xstFreq            1000
outputEnergies      50
outputPressure      50

# Fixed Atoms Constraint (set PDB beta-column to 1)
if {1} {
fixedAtoms          on
fixedAtomsFile      mineq-01.pdb
fixedAtomsCol       B
fixedAtomsForces    on
}

#############################################################
## EXECUTION SCRIPT                                        ##
#############################################################

# Minimization
if {1} {
minimize            1000
reinitvels          $temperature
}

run 500000;# 0.5 ns
""" %(bxsize, bysize,bzsize,bcenx, bceny, bcenz), file=open("mineq-01.conf", "w"))

In [31]:
csolec=np.unique(plm.get('resid', sel='resname %s and z > 0' %(csolv)))

## Calculate Cylinder Radius

In [32]:
radius=int(math.floor(np.sqrt((xsize*xsize)+(ysize*ysize)))/2)
print(radius)

29


## Calculate Cylinder Smaller Radius

In [33]:
if xsize > ysize:
        smallr=int(math.ceil(xsize)/2)
else:
        smallr=int(math.ceil(ysize)/2)
print(smallr)

24


## Create Collective Variable File for Eq02

In [34]:
print("""
Colvarstrajfrequency    50
Colvarsrestartfrequency 50
""", file=open("cosolvent.col", "w"))

for element in csolec:    
    zio=plm.get("serial",sel="resname %s and resid %s and z > 0" %(csolv,element))
    print("""
    ### Mol n. %s ###
    colvar {
       name ec_%s
       upperboundary %s
       upperwallconstant 200.0
       lowerboundary 0.0
       lowerwallconstant 200.0
       distancexy {
          ref {
             dummyAtom ( 0.000, 0.000, 0.000 )
          }
          main {
             atomNumbers {%s}
          }
         axis (0.0, 0.0, 1.0)
       }
    }

    colvar {
       name ecz_%s
       upperboundary %s
       lowerboundary 0.0
       upperwallconstant 200.0
       lowerwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( 0.000, 0.000, %s )
          }
          main {
            atomNumbers {%s}
          }
       }
    }
""" %(element,element,radius,str(zio).strip('[]'),element,zheight,zl2,str(zio).strip('[]')), file=open("cosolvent.col", "a"))

## Create Input File for Eq02

In [35]:
print("""
#############################################################
## JOB DESCRIPTION                                         ##
#############################################################

# Min. and Eq. of Co-Solvent Membrane System
# embedded in POPC membrane, ions and water.
# Protein constrained. PME, Constant Pressure.

#############################################################
## ADJUSTABLE PARAMETERS                                   ##
#############################################################

structure          structure.psf
coordinates        structure.pdb

outputName         mineq-02

set temperature    310

# Continuing a job from the restart files
if {1} {
set inputname      mineq-01
binCoordinates     ./$inputname.restart.coor
binVelocities      ./$inputname.restart.vel  
extendedSystem	   ./$inputname.restart.xsc
} 

firsttimestep      501000 ;# last step of previous run

#############################################################
## SIMULATION PARAMETERS                                   ##
#############################################################

# Input
paraTypeCharmm	    on
parameters          parameters

wrapWater           on
wrapAll             on

# Force-Field Parameters
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5

# Integrator Parameters
timestep            2.0  ;# 2fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2  
stepspercycle       20

#PME (for full-system periodic electrostatics)
if {1} {
PME                yes
PMEGridSpacing     1.0
}

# Constant Temperature Control
langevin            on    ;# do langevin dynamics
langevinDamping     1     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature

# Constant Pressure Control (variable volume)
if {1} {
useGroupPressure      yes ;# needed for 2fs steps
useFlexibleCell       yes  ;# no for water box, yes for membrane
useConstantArea       no  ;# no for water box, yes for membrane

langevinPiston        on
langevinPistonTarget  1.01325 ;#  in bar -> 1 atm
langevinPistonPeriod  200.
langevinPistonDecay   50.
langevinPistonTemp    $temperature
}

restartfreq        1000     ;# 1000steps = every 2ps
dcdfreq            1000
xstFreq            1000
outputEnergies      50
outputPressure      50

#############################################################
## EXTRA PARAMETERS                                        ##
#############################################################

# Put here any custom parameters that are specific to 
# this job (e.g., SMD, TclForces, etc...)

constraints on
consexp 2
consref ./structure.pdb
conskfile mineq-02.pdb       
conskcol B
margin 3

tclforces on
set waterCheckFreq              100
set lipidCheckFreq              100
set allatompdb                  ./structure.pdb
tclForcesScript                 keep_water_out.tcl

colvars on
colvarsConfig cosolvent.col

#############################################################
## EXECUTION SCRIPT                                        ##
#############################################################

# Minimization
if {1} {
minimize            500
reinitvels          $temperature
}

run 1000000 ;# 2.0 ns

""", file=open("mineq-02.conf", "w"))

## Check if there are gaps in the sequence and adjust the CA accordingly

In [36]:
# Copy cosolvent.col and append a line for the protein
dups = prot.copy()                       # Make a working copy
dups.filter("name CA and protein")      # Keep only C-alphas
rid = dups.get('resid')                 # Get list of residue ids
nrid, count= np.unique(rid,return_counts=True) # Get how many times each residue id appeared
nrid[count>1]                           # Show duplicates
deltas = np.diff(rid)
print(deltas)
new_rid = rid[:np.size(rid)-1] # no delta for last residue
new_rid[deltas!=1]
# Iterate over all residues (excluding last one)
rn = dups.get('resname')
for i in range(np.size(rid)-1):
    if(deltas[i]>1):
        print(rid[i],rn[i],' followed by ',rid[i+1],rn[i+1])
        print('%s-%s,'%(rid[i],rid[i+1]))
        print('%s-%s,%s-%s'%(rid[0],rid[i],rid[i+1],rid[-1]))
        ca=str('%s-%s,%s-%s'%(rid[0],rid[i],rid[i+1],rid[-1]))
        print(ca)

2018-05-08 17:37:06,522 - htmd.molecule.molecule - INFO - Removed 4502 atoms. 281 atoms remaining in the molecule.


[  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
 159   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   

In [37]:
ca=('%s-%s'%(nrid[0],nrid[-1]))
print(ca)

19-457


In [38]:
ca=('19-217,376-457')
print(ca)

19-217,376-457


## Create Collective Variable File for Eq03

In [39]:
shutil.copy2('cosolvent.col', 'cosolvent2.col')
print("""
colvar {
  name Origin
  distance {
     group1 {
      psfSegID PRO
      atomNameResidueRange CA %s
      centerReference on
      refPositions (0,0,0)
     }
     group2 {
     dummyAtom (%s,%s,%s)
     }
  }
}
""" %(ca,protcenx,protceny,protcenz), file=open("cosolvent2.col", "a"))

## Create Input File for Eq03

In [40]:
print("""
#############################################################
## JOB DESCRIPTION                                         ##
#############################################################

# Min. and Eq. of Co-Solvent Membrane System
# embedded in POPC membrane, ions and water.
# Protein released. PME, Constant Pressure.

#############################################################
## ADJUSTABLE PARAMETERS                                   ##
#############################################################

structure          structure.psf
coordinates        structure.pdb
outputName         eq-03

set temperature    310

# Continuing a job from the restart files
if {1} {
set inputname      mineq-02
binCoordinates     ./$inputname.restart.coor
binVelocities      ./$inputname.restart.vel  
extendedSystem	   ./$inputname.restart.xsc
} 

firsttimestep      1501500 ;# last step of previous run

#############################################################
## SIMULATION PARAMETERS                                   ##
#############################################################

# Input
paraTypeCharmm	    on
parameters          parameters

wrapWater           on
wrapAll             on

# Force-Field Parameters
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5

# Integrator Parameters
timestep            2.0  ;# 2fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2  
stepspercycle       20

#PME (for full-system periodic electrostatics)
if {1} {
PME                yes
PMEGridSpacing     1.0
}

# Constant Temperature Control
langevin            on    ;# do langevin dynamics
langevinDamping     1     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature

# Constant Pressure Control (variable volume)
if {1} {
useGroupPressure      yes ;# needed for 2fs steps
useFlexibleCell       yes  ;# no for water box, yes for membrane
useConstantArea       no  ;# no for water box, yes for membrane

langevinPiston        on
langevinPistonTarget  1.01325 ;#  in bar -> 1 atm
langevinPistonPeriod  200.
langevinPistonDecay   50.
langevinPistonTemp    $temperature
}

restartfreq        1000     ;# 1000steps = every 2ps
dcdfreq            1000
xstFreq            1000
outputEnergies      50
outputPressure      50

#############################################################
## EXTRA PARAMETERS                                        ##
#############################################################

# Put here any custom parameters that are specific to 
# this job (e.g., SMD, TclForces, etc...)

colvars on
colvarsConfig cosolvent2.col

#############################################################
## EXECUTION SCRIPT                                        ##
#############################################################

run 5000000;# 10.0 ns
""", file=open("eq-03.conf", "w"))

## Create Collective Variable File for Production

In [41]:
print("""
Colvarstrajfrequency    50
Colvarsrestartfrequency 50
""", file=open("cosolvent3.col", "w"))

for element in csolec:
    zio=plm.get("serial",sel="resname %s and resid %s and z > 0" %(csolv,element))
    print("""
    ### Mol n. %s ###
    colvar {
       name ec_%s
       upperboundary %s
       upperwallconstant 200.0
       lowerboundary 0.0
       lowerwallconstant 200.0
       distancexy {
          ref {
             dummyAtom ( 0.000, 0.000, 0.000 )
          }
          main {
             atomNumbers {%s}
          }
         axis (0.0, 0.0, 1.0)
       }
    }

    colvar {
       name ecz_%s
       upperboundary %s
       lowerboundary 0.0
       upperwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( 0.000, 0.000, %s )
          }
          main {
            atomNumbers {%s}
          }
       }
    } 
    """ %(element,element,smallr,str(zio).strip('[]'),element,zheight,zl2,str(zio).strip('[]')), file=open("cosolvent3.col", "a"))
print("""
    colvar {
      name Origin
      distance {
         group1 {
          psfSegID PRO
          atomNameResidueRange CA %s
          centerReference on
          refPositions (0,0,0)
         }
         group2 {
         dummyAtom (%s,%s,%s)
         }
      }
    }
    """%(ca,protcenx,protceny,protcenz), file=open("cosolvent3.col", "a"))

## Create Input File for Production

In [42]:
print("""
#############################################################
## JOB DESCRIPTION                                         ##
#############################################################

# Min. and Eq. of Co-Solvent Membrane System
# embedded in POPC membrane, ions and water.
# Protein released. PME, Constant Pressure and Area.

#############################################################
## ADJUSTABLE PARAMETERS                                   ##
#############################################################

structure          structure.psf
coordinates        structure.pdb
outputName         run          

set temperature    310

# Continuing a job from the restart files
if {1} {
set inputname      eq-03
binCoordinates     ../equil/$inputname.restart.coor
binVelocities      ../equil/$inputname.restart.vel  
extendedSystem	   ../equil/$inputname.restart.xsc
} 

firsttimestep      6501500 ;# last step of previous run

#############################################################
## SIMULATION PARAMETERS                                   ##
#############################################################

# Input
paraTypeCharmm	    on
parameters          parameters

wrapWater           on
wrapAll             on

# Force-Field Parameters
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5

# Integrator Parameters
timestep            2.0  ;# 2fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2  
stepspercycle       20

#PME (for full-system periodic electrostatics)
if {1} {
PME                yes
PMEGridSpacing     1.0
}

# Constant Temperature Control
langevin            on    ;# do langevin dynamics
langevinDamping     1     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature

# Constant Pressure Control (variable volume)
if {1} {
useGroupPressure      yes ;# needed for 2fs steps
useFlexibleCell       yes  ;# no for water box, yes for membrane
useConstantArea       yes ;# no for water box, yes for membrane

langevinPiston        on
langevinPistonTarget  1.01325 ;#  in bar -> 1 atm
langevinPistonPeriod  200.
langevinPistonDecay   50.
langevinPistonTemp    $temperature
}

restartfreq        1000     ;# 1000steps = every 2ps
dcdfreq            1000
xstFreq            1000
outputEnergies      50
outputPressure      50

#############################################################
## EXTRA PARAMETERS                                        ##
#############################################################

# Put here any custom parameters that are specific to 
# this job (e.g., SMD, TclForces, etc...)

constraints on
consexp 2 
consref ./structure.pdb
conskfile prod.pdb
conskcol B
margin 3  

colvars on
colvarsConfig cosolvent3.col

#############################################################
## EXECUTION SCRIPT                                        ##
#############################################################
if {1} { 
  minimize            240
  reinitvels          $temperature
  } 

  run 20000000 ;# 40.0 ns
""",file=open("run.conf", "w"))

## Print System Specs in a file

In [46]:
print("""
Protein Dimensions: %s %s
Cosolvent Boxes Dimensions: %s %s %s
Number of Cosolvent Molecules: %s
Cell Dimensions: %s %s %s
Cell Origin: %s %s %s
Protein Center of Mass: %s %s %s
Cylinder Radius: %s
Cylinder Z Boundary: %s
Cylinder Smaller Radius: %s
""" %(xsize,ysize,intxs,intys,intzs,cnumsol,bxsize,bysize,bzsize,bcenx,bceny,bcenz,protcenx,protceny,protcenz,radius,zl2,smallr),file=open("system-specs.txt", "w"))